In [11]:
#|default_exp maximdataset


In [12]:
#| export
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
import torch
from typing import Iterator, Optional, Sized
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import polars as pl
import pandas as pd
import os,gc
import numpy as np
from collections import OrderedDict
from sklearn.preprocessing import RobustScaler
from scipy.interpolate import interp1d
from fastai.vision.all import DataLoaders, OptimWrapper
from torch_geometric.nn.pool import knn_graph
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj


In [13]:
import numpy as np

def generate_mask(L):
    mask = np.random.choice([True, False], size=L, p=[0.93, 0.07])
    return mask

In [14]:
import sys
sys.path.append('/opt/slh/icecube/')
from icecube.models import EncoderWithDirectionReconstructionV10, EncoderWithDirectionReconstructionV11, EncoderWithDirectionReconstructionV13, EncoderWithDirectionReconstructionV12_V2, EncoderWithDirectionReconstructionV14, EncoderWithDirectionReconstructionV20, EncoderWithDirectionReconstructionV20_COMB
#from icecube.modelsgraph import PytorchEGNNV0
import matplotlib.pyplot as plt
from icecube.data_train_v3 import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3608879/3438902869.py:3 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3608879/3438902869.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'EncoderWithDirectionReconstructionV20_COMB' from 'icecube.models' 
(/opt/slh/icecube/icecube/models.py)

In [15]:
bs = 32
SELECTION = 'total'
NUM_WORKERS = 4
PATH = '../data/'
L = 192

ds_train = IceCubeCache(PATH, mode='train', L=L, selection=SELECTION,reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, selection=SELECTION, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=NUM_WORKERS, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, selection=SELECTION, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))


dls = DataLoaders(dl_train,dl_val)

In [16]:
x0 = next(iter(dls[0]))[0]

/opt/slh/icecube/icecube/data_train_v3.py:264: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)


In [ ]:
md = EncoderWithDirectionReconstructionV20_COMB(dim=384, dim_base=128, depth=8, head_size=32).eval().cuda()
with torch.no_grad():
    out = md(x0)

In [ ]:
out

tensor([[-0.2948, -0.0184, -0.1655],
        [-0.3153, -0.2677, -0.3560],
        [-0.1494, -0.1540, -0.2758],
        [-0.2310, -0.0754, -0.1838],
        [-0.3037, -0.1727, -0.1202],
        [-0.5200, -0.0719, -0.2045],
        [-0.2027, -0.2716, -0.4945],
        [-0.2864, -0.2059, -0.2357],
        [-0.1378, -0.1761, -0.4584],
        [-0.2995, -0.0389, -0.1954],
        [-0.2156, -0.2261, -0.2940],
        [-0.0449, -0.1183, -0.3248],
        [-0.3380, -0.0747, -0.3446],
        [-0.4523,  0.0046, -0.0233],
        [-0.2089, -0.2059, -0.2280],
        [-0.1817, -0.1839, -0.3230],
        [-0.3092, -0.1283, -0.1879],
        [-0.3377, -0.0625, -0.3530],
        [-0.3558, -0.1810, -0.3353],
        [-0.3499, -0.0530, -0.2381],
        [-0.2441, -0.2144, -0.2965],
        [-0.2146, -0.1647, -0.2634],
        [-0.2073, -0.1494, -0.2930],
        [-0.2132, -0.1480, -0.3498],
        [-0.1204, -0.3001, -0.3365],
        [-0.4720, -0.1019, -0.2659],
        [-0.3887, -0.0495, -0.1479],
 

In [ ]:
# mask = batch["mask"]
# Lmax = mask.sum(-1).max()
# fe = ExtractorV0(dim=256, dim_base=96).cuda()
# x = fe(batch, Lmax)
# mask = mask[:,:Lmax]
# #ptr = torch.cat([torch.zeros(1, dtype=torch.long, device=batch.device), mask.sum(1).cumsum(0)])
# x = x[mask]
# batch_index = mask.nonzero()[:, 0]
# pos = batch['pos'][:,:Lmax][mask]
# edge_index = knn_graph(
#                 x=pos,
#                 k=8,
#                 batch=batch_index,
#             ).to(mask.device)


In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()

In [ ]:
generate_mask(100).sum()

In [ ]:
import timm


In [ ]:
out

tensor([[-0.5899,  1.1641,  0.4085, -0.1833]])

In [ ]:
F.interpolate(out[0], size=(128,128), mode='bilinear', align_corners=False).shape

torch.Size([1, 64, 128, 128])

In [ ]:
adaptive_avgmax_pool2d(out[0])

torch.Size([1, 512])